# LangChain 핵심 메서드: invoke, batch, stream

이 노트북에서는 LangChain의 **세 가지 핵심 실행 메서드**를 알아봅니다.

## 메서드 개요

| 메서드 | 설명 | 반환값 | 사용 상황 |
|--------|------|--------|----------|
| `invoke()` | 단일 입력 처리 | 단일 결과 | 일반적인 1회 호출 |
| `batch()` | 여러 입력 동시 처리 | 결과 리스트 | 대량 처리, 병렬 실행 |
| `stream()` | 스트리밍 출력 | 토큰 제너레이터 | 실시간 응답 표시 |

## Runnable 인터페이스

LangChain의 모든 컴포넌트(LLM, Prompt, Parser, Chain)는 **Runnable** 인터페이스를 구현합니다.

```python
# 모든 Runnable 객체는 이 메서드들을 지원
runnable.invoke(input)      # 동기 실행
runnable.batch([inputs])    # 배치 실행
runnable.stream(input)      # 스트리밍

# 비동기 버전
await runnable.ainvoke(input)
await runnable.abatch([inputs])
async for chunk in runnable.astream(input)
```

---

# 1. Ollama 설치 및 서버 실행

In [ ]:
import subprocess
import time

# zstd 설치 (Ollama 설치의 사전 요구 사항)
!apt-get install -y zstd

# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# 백그라운드에서 Ollama 서버 실행
subprocess.Popen(['ollama', 'serve'])

time.sleep(3)

# 2. 모델 다운로드 & 패키지 설치

- `ollama pull llama3.2` - Llama 3.2 모델 다운로드
- `pip install langchain-ollama` - LangChain Ollama 통합 패키지 설치

In [ ]:
!ollama pull llama3.2
!pip install -q langchain-ollama

# 3. 모델 초기화

In [ ]:
from langchain_ollama import ChatOllama

model = ChatOllama(model='llama3.2')

# 4. invoke() - 단일 입력 처리

**가장 기본적인 실행 메서드**입니다. 하나의 입력을 받아 하나의 결과를 반환합니다.

```python
result = model.invoke(input)
```

**특징:**
- 동기(synchronous) 실행
- 응답이 완료될 때까지 대기
- 가장 간단하고 직관적인 방식

In [ ]:
# invoke() - 단일 입력, 단일 출력
response = model.invoke('반가워요!')

print("=== invoke() 결과 ===")
print(f"타입: {type(response).__name__}")
print(f"응답: {response.content}")

# 5. batch() - 여러 입력 동시 처리

**여러 입력을 한 번에 처리**하여 결과 리스트를 반환합니다.

```python
results = model.batch([input1, input2, input3])
```

**특징:**
- 내부적으로 **병렬 처리** (효율적)
- 순서대로 결과 반환 (입력 순서 = 출력 순서)
- 대량 데이터 처리에 적합

**옵션:**
```python
# 동시 실행 수 제한
results = model.batch(inputs, config={"max_concurrency": 5})
```

In [ ]:
# batch() - 여러 입력, 여러 출력
responses = model.batch(['반가워요!', '잘 있어요!', '좋은 하루 되세요!'])

print("=== batch() 결과 ===")
print(f"타입: {type(responses).__name__}")
print(f"결과 개수: {len(responses)}")
print()

for i, resp in enumerate(responses):
    print(f"[{i+1}] {resp.content[:50]}..." if len(resp.content) > 50 else f"[{i+1}] {resp.content}")

# 6. stream() - 스트리밍 출력

**응답을 토큰 단위로 실시간 수신**합니다. ChatGPT처럼 글자가 하나씩 나타나는 효과를 구현할 때 사용합니다.

```python
for chunk in model.stream(input):
    print(chunk.content, end='')
```

**특징:**
- **제너레이터(Generator)** 반환
- 첫 토큰이 생성되면 바로 수신 시작
- 사용자 경험(UX) 향상 - 대기 시간 체감 감소
- 긴 응답에서 특히 유용

In [ ]:
# stream() - 토큰 단위 스트리밍
print("=== stream() 결과 ===")
print("실시간 출력: ", end='')

for chunk in model.stream('잘 있어요!'):
    print(chunk.content, end='', flush=True)

print()  # 줄바꿈

# 7. 스트리밍 토큰 상세 확인

각 chunk가 어떤 내용을 담고 있는지 확인해봅니다.

In [ ]:
# 각 chunk 상세 확인
print("=== 각 chunk 내용 ===")

chunks = []
for i, chunk in enumerate(model.stream('안녕!')):
    chunks.append(chunk.content)
    print(f"chunk[{i}]: '{chunk.content}'")

print(f"\n전체 응답: {''.join(chunks)}")

---

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본 (OpenAI)
from langchain_openai.chat_models import ChatOpenAI
model = ChatOpenAI()

# 변경 (Ollama)
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2')
```

> `invoke()`, `batch()`, `stream()` 메서드는 동일하게 사용됩니다.

## 메서드 선택 가이드

| 상황 | 추천 메서드 |
|------|------------|
| 일반적인 단일 요청 | `invoke()` |
| 여러 문서 요약, 번역 등 | `batch()` |
| 챗봇 UI, 실시간 응답 | `stream()` |
| 비동기 웹 서버 (FastAPI 등) | `ainvoke()`, `astream()` |

## 체인에서도 동일하게 사용

```python
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("{topic}에 대해 설명해줘")
chain = prompt | model | StrOutputParser()

# 체인에서도 세 메서드 모두 사용 가능
chain.invoke({"topic": "AI"})
chain.batch([{"topic": "AI"}, {"topic": "ML"}])
for chunk in chain.stream({"topic": "AI"}):
    print(chunk, end='')
```

## 비동기 버전 (async)

```python
import asyncio

async def main():
    # 비동기 invoke
    result = await model.ainvoke('안녕!')
    
    # 비동기 batch
    results = await model.abatch(['안녕!', '반가워!'])
    
    # 비동기 stream
    async for chunk in model.astream('안녕!'):
        print(chunk.content, end='')

asyncio.run(main())
```